In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('./李高效生产及绿色防控技术.pdf')
splitter = RecursiveCharacterTextSplitter()
chunks = splitter.split_documents(loader.lazy_load())

In [3]:
import pandas as pd
df_chunks = pd.DataFrame({
  'text': [chunk.page_content for chunk in chunks],
  'source': [chunk.metadata['source'] for chunk in chunks],
  'page': [chunk.metadata['page'] for chunk in chunks]
})
df_chunks.to_csv('chunks.csv', sep="|", errors='ignore')

In [4]:

df_chunks = pd.read_csv('chunks.csv', sep="|", index_col=0)
df_chunks

,text,source,page
0,李产业发展现状\n一、中国李产业现状\n(一)栽培历史\n我国李栽培历史久远，原始居民就有采...,./李高效生产及绿色防控技术.pdf,0
1,李高效生产及绿色防控技术\n考证，“藉栽”指自根繁殖，即包括扦插、压条、分株等无性繁殖方\n...,./李高效生产及绿色防控技术.pdf,1
2,"第一章李产业发展现状\n年积温在4500~8000℃,最冷月平均气温0~15℃,是热带与温带...",./李高效生产及绿色防控技术.pdf,2
3,李高效生产及绿色防控技术\n(3)资源品种。本区主要栽培种为中国李，部分为欧洲李、杏李\n和...,./李高效生产及绿色防控技术.pdf,3
4,第一章李产业发展现状\n2绥棱红(北方1号)，它的母本和父本分别来自于小黄李和福摩萨，\n由...,./李高效生产及绿色防控技术.pdf,4
...,...,...,...
145,第五章李贮运保鲜与加工\n(5)预煮。预煮前用1.5%的食盐水护色，最后用清水淘洗1次。\n...,./李高效生产及绿色防控技术.pdf,145
146,李高效生产及绿色防控技术\n2.主要加工设施设备\n(1)清洗设备。鼓泡清洗机、毛刷清洗机、...,./李高效生产及绿色防控技术.pdf,146
147,第五章李贮运保鲜与加工\n4.产品质量要求\n(1)感官指标。具有该品种李加工后应有的正常颜...,./李高效生产及绿色防控技术.pdf,147
148,李高效生产及绿色防控技术\n(3)清洗。采用果蔬专用清洗设备清洗表面灰尘、腐叶、虫卵等\n污...,./李高效生产及绿色防控技术.pdf,148


In [5]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key='0a502a38484c564bb123ff9a27bc3d3c.KR9qWCf9jfjHeMmy')

In [10]:
SYSTEM_PROMPT_SUBG = """
你是一个知识图谱构建专家，用户会给你一段文本，你的任务是准确地从这段文本里提取出实体和它们之间的关系，
重点考虑与脆李生产技术相关的知识，比如施肥、剪枝、病虫害防治、物候期等等。

你可以按下面步骤进行：
1. 实体可能是人、物体、时间、地点、组织等，尽可能保证实体的原子性和多样性。
2. 一个实体可能与多个实体有关系，尽可能多地提取出关系。
3. 将提取出的实体和关系按照如下三元组格式输出，只需要输出结果，不需要解释思路。
   足球|属于|球类运动
   人|踢|足球
   疫苗|预防|疾病
"""
def subg(text: str):
  res = client.chat.completions.create(
    model="glm-3-turbo",
    messages=[
      {"role": "system", "content": SYSTEM_PROMPT_SUBG},
      {"role": "user", "content": text},
    ]
  )
  return res.choices[0].message

In [11]:
from tqdm import tqdm

with open("edges-0-149.2.csv", "w+") as f:
  for i, row in tqdm(df_chunks.iterrows()):
    if 0 <= i < 150:
      content = subg(row["text"]).content
      lines = content.split("\n")
      for line in lines:
        f.write(f"{line}|{i}\n")
      f.flush()

150it [20:13,  8.09s/it]


In [14]:
with open('edges.csv', 'w') as w:
    with open('edges-0-149.2.csv') as r:
        for line in r:
            cells = line.split("|")
            if len(cells) != 4:
                continue
            w.write(line)

In [30]:
df_all = pd.read_csv("edges-2.5k.csv", sep="|", names=["u", "e", "v", "d"]).dropna()
df_all

,u,e,v,d
0,李树,种植,新石器时代,0
1,李树,种植,战国时代,0
2,李核,遗存,考古发掘,0
3,李果,采食,原始居民,0
4,李树,栽培,先民,0
...,...,...,...,...
2588,正辛醇,属于,李果干芳香成分,133
2589,己醛,属于,李果干芳香成分,133
2590,壬醛,属于,李果干芳香成分,133
2591,壬烯醛,属于,李果干芳香成分,133


In [31]:
nodes: set[str] = set()
links: dict[str, int] = dict()
for i, row in df_all.iterrows():
    u = row["u"]
    v = row["v"]
    e = row["e"]
    d = row["d"]
    nodes.add(u)
    nodes.add(v)
    k = f'{u}|{v}|{e}'
    if k not in links:
        links[k] = 0
    links[k] += 1

In [32]:
len(nodes)

2530

In [38]:
import random

data = {
    'nodes': [],
    'links': []
}
for node in list(nodes):
    data['nodes'].append({
        'id': node,
        'group': random.choice([0,1,2,3,4]),
    })
for k, w in links.items():
    u, v, e = k.split('|')
    data['links'].append({
        'source': u,
        'target': v,
        'value': w,
        'label': e,
    })
data

{'nodes': [{'id': '通过带毒的繁殖材料', 'group': 3},
  {'id': '广西大学农学院', 'group': 1},
  {'id': '花芽分化', 'group': 2},
  {'id': '苹果、梨、桃、李、杏、山楂、枣', 'group': 4},
  {'id': '粘虫板', 'group': 1},
  {'id': '保持土壤湿润', 'group': 2},
  {'id': '振落捕杀成虫', 'group': 4},
  {'id': '虫道', 'group': 2},
  {'id': '药剂可选用25%甲霜灵可湿性粉剂500倍液，或50%福美双可湿性粉剂500倍液，或45%代森铵水剂1000倍液',
   'group': 1},
  {'id': '湿度', 'group': 2},
  {'id': '贮藏期', 'group': 2},
  {'id': '初产椭圆形', 'group': 1},
  {'id': '霉层', 'group': 1},
  {'id': '子囊菌亚门', 'group': 3},
  {'id': '戊唑·噻唑锌悬浮剂', 'group': 3},
  {'id': '幸运李', 'group': 0},
  {'id': '降水量', 'group': 0},
  {'id': '无霜期', 'group': 1},
  {'id': '衰老树', 'group': 4},
  {'id': '19~40个', 'group': 3},
  {'id': '生长势旺的枝条', 'group': 2},
  {'id': '货堆', 'group': 4},
  {'id': '香气成分含量逐渐升高', 'group': 4},
  {'id': '广', 'group': 2},
  {'id': '竞争枝', 'group': 0},
  {'id': '土肥水管理', 'group': 0},
  {'id': '担子菌亚门', 'group': 3},
  {'id': '鞭状', 'group': 0},
  {'id': '734份', 'group': 3},
  {'id': '生长习性', 'group': 1},
  {'id': '巫山青脆

In [39]:
import json

with open('g.json', 'w') as w:
    json.dump(data, w, ensure_ascii=False)